In [1]:
import pandas as pd
import numpy as np
import sys, os
import gzip, json

In [2]:
df_reviews = pd.read_json("/Users/hyoon/Desktop/review-Hawaii_10.json", lines=True)
df_reviews.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.139654e+20,manuel grimaldo,1591839903487,5,Great new upgrade,None,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e
1,1.166558e+20,Enrique Lara,1568059018979,5,None,None,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e
2,1.008341e+20,Gregory Donaldson,1594885588335,5,None,None,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e
3,1.032072e+20,Brian Baker,1575951131613,5,None,None,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e
4,1.085262e+20,Kam J,1573076723916,3,None,None,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e


In [3]:
df_metadata = pd.read_json("/Users/hyoon/Desktop/meta-Hawaii.json", lines=True)
df_metadata.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,[Restaurant],4.4,18,None,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 11AM,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...
1,SMP - Single Marine Program,"SMP - Single Marine Program, G St, Kailua, HI ...",0x7c00159b5b1b1d25:0x8d2d85d4a758290e,None,21.440345,-157.754347,[Recreation center],4.1,18,None,"[[Thursday, 8AM–9PM], [Friday, 8AM–9PM], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Opens soon ⋅ 8AM,"[0x7c00155df23846af:0xfbe051d208292028, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...
2,2 Cheesy Guys,"2 Cheesy Guys, 1486 HI-30, Wailuku, HI 96793",0x7954d376a8b12db3:0xa51dd57e1cc14ca9,None,20.853014,-156.503671,[Food court],5.0,6,None,"[[Thursday, Closed], [Friday, 11AM–6PM], [Satu...","{'Service options': ['Curbside pickup', 'Takeo...",Closed ⋅ Opens 11AM Fri,None,https://www.google.com/maps/place//data=!4m2!3...
3,Kraken Coffee Kahului,"Kraken Coffee Kahului, 520 Keolani Pl, Kahului...",0x7954d370921ff6bd:0x3193ba783e26d032,None,20.888238,-156.450614,[Coffee shop],4.8,8,$,"[[Thursday, 6:30AM–7PM], [Friday, 6:30AM–7PM],...","{'Service options': ['Curbside pickup', 'No-co...",Open ⋅ Closes 7PM,None,https://www.google.com/maps/place//data=!4m2!3...
4,Akasatana Ramen Kyoto,"Akasatana Ramen Kyoto, 1450 Ala Moana Blvd, Ho...",0x7c006df045b01715:0xe945c308688e1a46,None,21.290463,-157.843730,[Ramen restaurant],5.0,1,None,"[[Thursday, 11AM–8:30PM], [Friday, 11AM–8:30PM...","{'Service options': ['Takeout', 'Dine-in', 'De...",Closed ⋅ Opens 11AM,"[0x7c006df018f6177d:0x9beb6db40fadcb2, 0x7c006...",https://www.google.com/maps/place//data=!4m2!3...


In [ ]:
# only necessary columns for our policy
df_reviews_reduced = df_reviews[["user_id", "time", "rating", "text", "gmap_id"]].copy()
df_metadata_reduced = df_metadata[["gmap_id", "name", "category", "description"]].copy()

In [5]:
df_merged = pd.merge(df_reviews_reduced, df_metadata_reduced, on="gmap_id", how="left")
df_merged

,user_id,time,rating,text,gmap_id,name,category,description
0,1.139654e+20,1591839903487,5,Great new upgrade,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
1,1.139654e+20,1591839903487,5,Great new upgrade,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
2,1.166558e+20,1568059018979,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
3,1.166558e+20,1568059018979,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
4,1.008341e+20,1594885588335,5,None,0x7c00159b5b1b1d25:0x8d2d85d4a758290e,SMP - Single Marine Program,[Recreation center],None
...,...,...,...,...,...,...,...,...
1505006,1.019840e+20,1523148228376,5,None,0x7953b4a4114e37f7:0x374b5a1f84f48a1a,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...
1505007,1.130985e+20,1578633104357,5,None,0x7953b4a4114e37f7:0x374b5a1f84f48a1a,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...
1505008,1.165499e+20,1578120354635,5,None,0x7953b4a4114e37f7:0x374b5a1f84f48a1a,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...
1505009,1.011753e+20,1552474937379,4,None,0x7953b4a4114e37f7:0x374b5a1f84f48a1a,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...


In [6]:
#rename columns, create new columsn, drop unneccesary columns
df_merged_processed = df_merged.rename(columns={"name":"business_name", "category":"business_category", "description":"business_description"})
df_merged_processed["_id"] = df_merged_processed["user_id"].astype(str) + "_" + df_merged_processed["time"].astype(str)
df_merged_processed = df_merged_processed.drop(columns=["user_id", "time", "gmap_id"])
df_merged_processed

,rating,text,business_name,business_category,business_description,_id
0,5,Great new upgrade,SMP - Single Marine Program,[Recreation center],None,1.1396541707957663e+20_1591839903487
1,5,Great new upgrade,SMP - Single Marine Program,[Recreation center],None,1.1396541707957663e+20_1591839903487
2,5,None,SMP - Single Marine Program,[Recreation center],None,1.1665581913729334e+20_1568059018979
3,5,None,SMP - Single Marine Program,[Recreation center],None,1.1665581913729334e+20_1568059018979
4,5,None,SMP - Single Marine Program,[Recreation center],None,1.0083411999455008e+20_1594885588335
...,...,...,...,...,...,...
1505006,5,None,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.019839942414649e+20_1523148228376
1505007,5,None,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.1309851064275971e+20_1578633104357
1505008,5,None,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.1654991911971888e+20_1578120354635
1505009,4,None,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.0117532049244978e+20_1552474937379


In [7]:
# remove duplicate _id and rows with null values or non english character
df_merged_drop_dup = df_merged_processed.drop_duplicates(subset=["_id"])
df_merged_final = df_merged_drop_dup[~df_merged_drop_dup.isnull().any(axis=1)]

allowed_pattern = r'^[A-Za-z0-9\s.,!?;:\'\"()\[\]\{\}<>-]*$'
df_merged_final = df_merged_final[(df_merged_final['text'].str.match(allowed_pattern, na=False))
                                  &(df_merged_final['business_name'].str.match(allowed_pattern, na=False))
                                  #&(df_merged_final['business_category'].str.match(allowed_pattern, na=False))
                                  &(df_merged_final['business_description'].str.match(allowed_pattern, na=False))]
df_merged_final

,rating,text,business_name,business_category,business_description,_id
180,5,My husband took me here for my birthday! The ...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.089786951023629e+20_1507964592497
182,4,was a great place. Now closed. Too bad. We'll...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.124085651879834e+20_1519880201044
184,5,"cozy, great food. love how you can sign your b...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1571615809204435e+20_1490033397528
186,4,"Not your average little Italian joint, don't e...",Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.181306411818766e+20_1511165172600
188,3,Service was great and the mozzarella fritta an...,Buona Sera,"[Italian restaurant, Restaurant]",Casual trattoria serving familiar Italian entr...,1.1430211684992369e+20_1505179108178
...,...,...,...,...,...,...
1504874,5,Nice,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.1626263862963937e+20_1490149857916
1504885,4,(Translated by Google) Unfortunately when we w...,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.0126644436078756e+20_1515420692947
1504887,3,"(Translated by Google) Beautiful waterfall, Un...",Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.0436870989554486e+20_1533527358096
1504890,4,(Translated by Google) Not a rainbow to be see...,Rainbow Falls,[Waterfall],Rainforest waterfall featuring distinctive rai...,1.1601463698182888e+20_1533372880422


In [ ]:
#df_merged_final.to_csv("/Users/hyoon/Desktop/final_data_total.csv")

In [9]:
#extract N reviews per business_name for finalized train_validation dataset of 10000 entries.
total_samples = 10000
n_groups = df_merged_final['business_name'].nunique()
samples_per_group = total_samples // n_groups

df_sampled = df_merged_final.groupby('business_name', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), samples_per_group), random_state=42)
)

if len(df_sampled) < total_samples:
    remaining = total_samples - len(df_sampled)
    df_remaining = df_merged_final.drop(df_sampled.index).sample(n=remaining, random_state=42)
    df_sampled = pd.concat([df_sampled, df_remaining])

print(len(df_sampled))
print(df_sampled['business_name'].value_counts())

10000
business_name
Starbucks                                                          168
Pearl Harbor Historic Sites Visitor Center                          97
Battleship Missouri Memorial                                        76
Denny's                                                             71
Aloha Stadium Swap Meet                                             70
                                                                  ... 
Japengo Waikiki                                                      4
Sadie Seymour Botanical Gardens and the Kona Educational Center      4
Guava Shop                                                           3
Allsaints                                                            3
SoHa Living Kahala                                                   2
Name: count, Length: 424, dtype: int64


/var/folders/0r/chgngwln0ns7swt43699fqpm0000gn/T/ipykernel_21987/353118156.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_merged_final.groupby('business_name', group_keys=False).apply(


In [ ]:
#df_sampled.to_csv("final_data_sampled.csv")